In [1]:
import tensorflow as tf
import numpy as np

In [6]:
def input_fn():
    # repeat(step_repeat_number)
    dataset = tf.data.TextLineDataset('./up_down_dataset.csv')\
        .batch(20)\
        .repeat(9999)\
        .make_one_shot_iterator()\
        .get_next()
    lines = tf.decode_csv(dataset, record_defaults=[[0]]*11)
    feature = tf.stack(lines[:-1], axis=1)
    label = tf.expand_dims(lines[-1], axis=-1)
    
    feature = tf.cast(feature, tf.float32)
    label = tf.cast(label, tf.float32)
    return {'feature':feature}, label

In [9]:
def model_fn(features, labels, mode):  
    # estimator에서 원하는 모양이 세가지 파라미터를 똑같은 이름으로 받아야한다.
    
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PRED = mode == tf.estimator.ModeKeys.PREDICT
    
    # input layer
    layer1 = tf.layers.dense(features['feature'], units=10, activation=tf.nn.relu) # units = the number of node
    
    # hidden layers
    layer2 = tf.layers.dense(layer1, units=10, activation=tf.nn.relu)
    layer3 = tf.layers.dense(layer2, units=10, activation=tf.nn.relu)
    layer4 = tf.layers.dense(layer3, units=10, activation=tf.nn.relu)
    
    # output layer
    out = tf.layers.dense(layer4, units=1)  # label = 0 or 1
    
    if TRAIN:
        gs = tf.train.get_global_step()
        # loss function
        loss = tf.losses.sigmoid_cross_entropy(labels, out)
        train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss, global_step=gs)
        return tf.estimator.EstimatorSpec(
            mode=mode,loss=loss,train_op=train_op)
        
    elif EVAL:
        loss = tf.losses.sigmoid_cross_entropy(labels, out)
        # prediction
        pred = tf.nn.sigmoid(out)  # number of 0~1 
        # accuracy
        acc = tf.metrics.accuracy(labels, tf.round(pred))
        return tf.estimator.EstimatorSpec(
            mode=mode,loss=loss,eval_metric_ops={'acc':acc})
    elif PRED:
        prob = tf.nn.sigmoid(out)
        class_ = tf.round(prob)
        return tf.estimator.EstimatorSpec(mode=mode, predictions={'prob':prob,'class':class_})

In [23]:
if __name__ =='__main__':  # 이렇게 사용하는걸 적극 권장한다.
    
    # estimator 내부에서 생성되는 로그를 보기위한 것, jupyter에서 실행하면 바로나오더라.
    tf.logging.set_verbosity(tf.logging.INFO)  
    
    estimate = tf.estimator.Estimator(model_fn)
    estimate.train(input_fn, steps=1000)
    
    estimate.evaluate(input_fn,steps=10)
    
    
    data1 = np.array([1,2,3,4,5,6,7,8,9,0], np.float32)
    data2 = np.array([5,5,5,5,5,5,5,5,5,5], np.float32)
    data3 = np.array([9-i for i in range(10)], np.float32)
    data = np.stack([data1,data2,data3])
    
    pred_input_fn = tf.estimator\
        .inputs.numpy_input_fn({'feature':data},shuffle=False)
    for d, pred in zip(data, estimate.predict(pred_input_fn)):
        print('feature: {},prob: {}, class: {}'.format(d, pred['prob'], pred['class']))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpk9ii877c', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc61c45d8d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph wa